In [1]:
!pip install hmmlearn

     |████████████████████████████████| 153kB 2.8MB/s 
  Created wheel for hmmlearn: filename=hmmlearn-0.2.2-cp36-cp36m-linux_x86_64.whl size=326264 sha256=69886cb027dd418e8f7aa338c39f8e4679b1d8bc2917019957272630d6922140
  Stored in directory: /root/.cache/pip/wheels/2c/b6/0e/63a865a30e21e01d04f417d8995fbfb793d6bd464707efc546
Successfully built hmmlearn


In [115]:
from collections import defaultdict

train_data = """'js' const enzyme = require ( 's' ) ; const Adapter = require ( 's' ) ; enzyme . configure ( { adapter : new Adapter ( ) } ) ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O $any$ O $any$ O O $any$ O O $any$ O O O O O
'js' const ForkTsCheckerWebpackPlugin = require ( 's' ) ; module . exports = { devtool : 's' , module : { rules : [ { test : "s" , loaders : [ 's' , 's' , 's' ] , } , { test : "s" , exclude : "s" , loaders : 's' , options : { transpileOnly : true , } , } , ] , } , resolve : { extensions : [ 's' , 's' , 's' , 's' ] , } , plugins : [ new ForkTsCheckerWebpackPlugin ( ) ] , } ;	O O $any$ O $any$ O O O O $complex$ O $complex$ O O $string$ O O O $complex$ O O $complex$ O O O $RegExp$ O O O $string[]$ O O O O O O O O O O O O $RegExp$ O O O $RegExp$ O O O $string$ O O O $complex$ O O $boolean$ O O O O O O O O O O O $complex$ O O $string[]$ O O O O O O O O O O O O O $any[]$ O O O $any$ O O O O O O
'js' const path = require ( 's' ) ; const docgen = require ( 's' ) ; const docgenParser = require ( 's' ) ; const webpackConfig = require ( 's' ) ; const pkg = require ( 's' ) ; const componentsWithoutPathLine = [ 's' ] ; const getComponentPathLine = componentPath => { const ext = path . extname ( componentPath ) ; const name = path . basename ( componentPath , ext ) ; if ( componentsWithoutPathLine . indexOf ( name ) >= 0 ) { return 's' ; } const dir = path . dirname ( componentPath ) . replace ( "s" , 's' ) ; return `template` ; } ; const root = path . resolve ( __dirname , 's' ) ; module . exports = { title : `template` , sections : [ { name : 's' , content : path . resolve ( root , 's' ) , components : path . resolve ( root , 's' ) , } , { name : 's' , content : path . resolve ( root , 's' ) , components : path . resolve ( root , 's' ) , } , { name : 's' , content : path . resolve ( root , 's' ) , components : path . resolve ( root , 's' ) , } , ] , ignore : [ 's' , 's' , 's' , 's' , 's' , 's' , 's' , 's' , 's' , 's' , ] , skipComponentsWithoutExample : true , resolver : docgen . resolver . findAllComponentDefinitions , propsParser : docgenParser . parse , getComponentPathLine , webpackConfig , assetsDir : path . resolve ( __dirname , 's' ) , context : { action : path . resolve ( __dirname , 's' ) , user : path . resolve ( __dirname , 's' ) , group : path . resolve ( __dirname , 's' ) , file : path . resolve ( __dirname , 's' ) , } , require : [ path . resolve ( root , 's' ) ] , serverPort : 0 , styleguideDir : path . resolve ( root , 's' ) , styleguideComponents : { Wrapper : path . resolve ( root , 's' ) , } , } ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $complex$ O $any$ O O O O O $any$ O $any$ O O O O O $string[]$ O O O O O O $string$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O O O O $string[]$ O $number$ O $any$ O O O O O O O O O O $any$ O $any$ O $any$ O $any$ O O $any$ O O O O O O O O O O O O $any$ O $any$ O $any$ O $any$ O O O O O O O O O $string$ O O O $complex$ O O O $string$ O O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O O O O $string$ O O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O O O O $string$ O O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O O O O O $string[]$ O O O O O O O O O O O O O O O O O O O O O O O O $boolean$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $string$ O $complex$ O $any$ O $any$ O $any$ O $any$ O O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O O O $any[]$ O O $any$ O $any$ O $any$ O O O O O $number$ O O O $any$ O $any$ O $any$ O $any$ O O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O O O O O O O O
'js' const path = require ( 's' ) ; const config = require ( 's' ) ; module . exports = Object . assign ( { } , config , { showSidebar : false , require : config . require . concat ( [ path . resolve ( __dirname , 's' ) ] ) , propsParser : ( ) => ( { } ) , theme : { color : { border : 's' , } , } , } ) ;	O O $any$ O $any$ O O O O O O O $any$ O O O O O O O O $ObjectConstructor$ O O O O O O O O O $false$ O O O $any[]$ O O O $any[]$ O $complex$ O O $any$ O $any$ O $any$ O O O O O O ${}$ O O O O O O O O O $complex$ O O $complex$ O O $string$ O O O O O O O O O O
'js' module . exports = function ( msg ) { return function ( ) { console . log ( msg , arguments ) ; } ; } ;	O $void$ O $void$ O O O $any$ O O O O O O O $Console$ O $void$ O $any$ O $IArguments$ O O O O O O
'js' const path = require ( 's' ) ; module . exports = { url : `template` , dir : 's' , viewports : { desktop : { width : 0 , height : 0 , deviceScaleFactor : 0 , } , } , launchOptions : { args : [ 's' , 's' , 's' ] , slowMo : 0 , } , navigationOptions : { waitUntil : 's' , } , } ;	O O $any$ O $any$ O O O O $complex$ O $complex$ O O $string$ O O O $string$ O O O $complex$ O O $complex$ O O $number$ O O O $number$ O O O $number$ O O O O O O O $complex$ O O $string[]$ O O O O O O O O O $number$ O O O O O $complex$ O O $string$ O O O O O O O
'js' const path = require ( 's' ) ; const fs = require ( 's' ) ; const util = require ( 's' ) ; const { template } = require ( 's' ) ; const HTMLtoJSX = require ( 's' ) ; const { JSDOM } = require ( 's' ) ; const writeFile = util . promisify ( fs . writeFile ) ; const readFile = util . promisify ( fs . readFile ) ; const converter = new HTMLtoJSX ( { createClass : false } ) ; async function generateIndex ( svgNames , indexTemplate , destPath ) { const contents = template ( indexTemplate ) ( { svgNames } ) ; const destIndexPath = path . resolve ( destPath , 's' ) ; console . log ( `template` ) ; await writeFile ( destIndexPath , contents , 's' ) ; } async function generateComponent ( svg , svgTemplate , templateData , destFilePath , optimizer ) { try { const svgContents = await readFile ( svg , 's' ) ; const optimizedsvgContents = await optimizer . optimize ( svgContents . toString ( ) ) ; const { window } = new JSDOM ( optimizedsvgContents . data ) ; const { innerHTML } = window . document . querySelector ( 's' ) ; const name = path . basename ( svg , path . extname ( svg ) ) ; const jsx = converter . convert ( innerHTML ) . replace ( 's' , 's' ) . replace ( 's' , 's' ) . trim ( ) ; const dirtyClassContents = template ( svgTemplate ) ( { name , jsx , ... templateData , } ) ; const classContents = dirtyClassContents . replace ( "s" , 's' ) . replace ( "s" , 's' ) ; console . log ( `template` ) ; await writeFile ( destFilePath , classContents , 's' ) ; } catch ( e ) { console . error ( e ) ; } } module . exports = { generateComponent , generateIndex , } ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O O $any$ O O $any$ O O O O O $any$ O $any$ O O O O O O $any$ O O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O O O $any$ O O $any$ O O $boolean$ O O O O O O O $Promise<void>$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O O O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O $Console$ O $void$ O O O O O $any$ O $any$ O $any$ O O O O O O O $Promise<void>$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O O $any$ O $any$ O O O O O $any$ O O $any$ O $any$ O $any$ O $any$ O O O O O O $any$ O O O $any$ O $any$ O $any$ O O O O $any$ O O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O $any$ O O O O O O $any$ O O O O O O $any$ O O O O $any$ O $any$ O $any$ O O O $any$ O $any$ O O $any$ O O O O O $any$ O $any$ O $any$ O O O O O O $any$ O O O O O O $Console$ O $void$ O O O O O $any$ O $any$ O $any$ O O O O O O O $any$ O O $Console$ O $void$ O $any$ O O O O $complex$ O $complex$ O O $Promise<void>$ O $Promise<void>$ O O O
'js' const fs = require ( 's' ) ; const path = require ( 's' ) ; const util = require ( 's' ) ; const glob = require ( 's' ) ; const SVGO = require ( 's' ) ; const { generateComponent , generateIndex } = require ( 's' ) ; const svgoConfig = require ( 's' ) ; const sourcePath = path . resolve ( __dirname , 's' ) ; const destPath = path . resolve ( __dirname , 's' ) ; const indexTemplatePath = path . resolve ( __dirname , 's' ) ; const iconTemplatePath = path . resolve ( __dirname , 's' ) ; const readFile = util . promisify ( fs . readFile ) ; const globFiles = util . promisify ( glob ) ; const optimizer = new SVGO ( svgoConfig ) ; function getSvgNames ( icons ) { return icons . map ( icon => path . basename ( icon , path . extname ( icon ) ) ) ; } ( async ( ) => { const indexTemplate = await readFile ( indexTemplatePath ) ; const iconTemplate = await readFile ( iconTemplatePath ) ; const icons = await globFiles ( `template` , { } ) ; const svgNames = getSvgNames ( icons ) ; await generateIndex ( svgNames , indexTemplate , destPath ) ; icons . forEach ( icon => { const name = path . basename ( icon , path . extname ( icon ) ) ; const destIconPath = path . resolve ( destPath , `template` ) ; generateComponent ( icon , iconTemplate , { } , destIconPath , optimizer ) ; } ) ; } ) ( ) ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O O $Promise<void>$ O $Promise<void>$ O O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O O O $any$ O O $any$ O $any$ O O O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O O O O O O O O $any$ O O $any$ O $any$ O O O $any$ O O $any$ O $any$ O O O $any$ O O $any$ O O O O O O O O $any$ O $any$ O $any$ O O O $Promise<void>$ O $any$ O $any$ O $any$ O O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O $Promise<void>$ O $any$ O $any$ O O O O $any$ O $any$ O O O O O O O O O O
'js' const fs = require ( 's' ) ; const path = require ( 's' ) ; const util = require ( 's' ) ; const glob = require ( 's' ) ; const SVGO = require ( 's' ) ; const { generateComponent , generateIndex } = require ( 's' ) ; const svgoConfig = require ( 's' ) ; const sourcePath = path . resolve ( __dirname , 's' ) ; const destPath = path . resolve ( __dirname , 's' ) ; const indexTemplatePath = path . resolve ( __dirname , 's' ) ; const svgTemplatePath = path . resolve ( __dirname , 's' ) ; const readFile = util . promisify ( fs . readFile ) ; const globFiles = util . promisify ( glob ) ; function getSizeFromFilePath ( filePath ) { const sizeRegex = "s" ; const sizeResults = filePath . match ( sizeRegex ) ; if ( sizeResults === null ) { throw new Error ( `template` ) ; } return sizeResults [ 0 ] ; } function getSvgNames ( illustrations ) { return illustrations . map ( illustration => { const size = getSizeFromFilePath ( illustration ) ; const filename = path . basename ( illustration , path . extname ( illustration ) ) ; return `template` ; } ) ; } ( async ( ) => { const indexTemplate = await readFile ( indexTemplatePath ) ; const svgTemplate = await readFile ( svgTemplatePath ) ; const svgs = await globFiles ( `template` , { } ) ; const svgNames = getSvgNames ( svgs ) ; await generateIndex ( svgNames , indexTemplate , destPath ) ; svgs . forEach ( svg => { const size = getSizeFromFilePath ( svg ) ; const templateData = { size } ; const name = path . basename ( svg , path . extname ( svg ) ) ; const destFilePath = path . resolve ( destPath , `template` ) ; const svgoConfigExtension = { cleanupIDs : { prefix : `template` , } , } ; const svgoFileConfig = svgoConfig ; svgoFileConfig . plugins . push ( svgoConfigExtension ) ; const optimizer = new SVGO ( svgoFileConfig ) ; generateComponent ( svg , svgTemplate , templateData , destFilePath , optimizer ) ; } ) ; } ) ( ) ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O O O O O O $Promise<void>$ O $Promise<void>$ O O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O O O $any$ O $any$ O O O $RegExp$ O O O O $any$ O $any$ O $any$ O $RegExp$ O O O O $any$ O O O O O O $ErrorConstructor$ O O O O O O $any$ O O O O O O $any$ O $any$ O O O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O O O O O O O O O O O O O $any$ O O $any$ O $any$ O O O $any$ O O $any$ O $any$ O O O $any$ O O $any$ O O O O O O O O $any$ O $any$ O $any$ O O O $Promise<void>$ O $any$ O $any$ O $any$ O O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O O O O $complex$ O O $complex$ O O $string$ O O O O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O $complex$ O O O $any$ O O $any$ O $any$ O O $Promise<void>$ O $any$ O $any$ O $complex$ O $any$ O $any$ O O O O O O O O O O
'js' const rename = require ( 's' ) ; const Generator = require ( 's' ) ; module . exports = class extends Generator { paths ( ) { this . destinationRoot ( 's' ) ; } prompting ( ) { const prompts = [ { type : 's' , name : 's' , message : 's' , } , ] ; return this . prompt ( prompts ) . then ( props => { const { name } = props ; const className = `template` ; this . props = { name , className } ; } ) ; } writing ( ) { const { name } = this . props ; this . registerTransformStream ( rename ( file => { file . basename = file . basename . replace ( 's' , name ) ; return file ; } ) , ) ; this . fs . copyTpl ( this . templatePath ( 's' ) , this . destinationPath ( `template` ) , this . props ) ; } } ;	O O $any$ O $any$ O O O O O $any$ O $any$ O O O O $any$ O $any$ O O O $any$ O $void$ O O O O O $any$ O O O O O $any$ O O O O $complex$ O O O $string$ O O O $string$ O O O $string$ O O O O O O O O O O $any$ O $complex$ O O $any$ O $any$ O O O O $any$ O O $any$ O O $string$ O O O O O $complex$ O O $any$ O $string$ O O O O O O $void$ O O O O O $any$ O O O O $complex$ O O O $any$ O $any$ O $any$ O O $any$ O $any$ O $any$ O $any$ O $any$ O O O $any$ O O O $any$ O O O O O O O O $any$ O $any$ O O O $any$ O O O O O O $any$ O O O O O O $complex$ O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum < %= name % > Type { BLACK = 's' , WHITE = 's' , }	O O O O $never$ O O $any$ O $any$ O O O $any$ O O O O
'js' const path = require ( 's' ) ; module . exports = ( ) => ( { parser : 's' , plugins : { 's' : { root : path . resolve ( __dirname ) , path : [ 's' ] , } , precss : { } , } , } ) ;	O O $any$ O $any$ O O O O $complex$ O $complex$ O O O O O O $string$ O O O $complex$ O O O O O $any$ O $any$ O $any$ O $any$ O O $string[]$ O O O O O O O ${}$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum AvatarSize { XLARGE = 's' , LARGE = 's' , MEDIUM = 's' , SMALL = 's' , XSMALL = 's' , }	O O $any$ O $AvatarSize.XLARGE$ O O O $AvatarSize.LARGE$ O O O $AvatarSize.MEDIUM$ O O O $AvatarSize.SMALL$ O O O $AvatarSize.XSMALL$ O O O O
export enum GutterSize { XXLARGE = 's' , XLARGE = 's' , LARGE = 's' , MEDIUM = 's' , SMALL = 's' , XSMALL = 's' , NONE = 's' , }	O O $any$ O $GutterSize.XXLARGE$ O O O $GutterSize.XLARGE$ O O O $GutterSize.LARGE$ O O O $GutterSize.MEDIUM$ O O O $GutterSize.SMALL$ O O O $GutterSize.XSMALL$ O O O $GutterSize.NONE$ O O O O
import { IPalette , ISemanticColors } from 's' ; const pond = 's' ; const lake = 's' ; const newBlue = 's' ; const river = 's' ; const bay = 's' ; const white = 's' ; const popRock = 's' ; const altRock = 's' ; const indieRock = 's' ; const punkRock = 's' ; const heavyMetal = 's' ; const deathMetal = 's' ; const blackMetal = 's' ; const lestat = 's' ; const dracula = 's' ; export const palette < IPalette > = { themeDarker : bay , themeDark : river , themeDarkAlt : newBlue , themePrimary : lake , themeLighter : pond , neutralDark : indieRock , neutralPrimary : blackMetal , neutralPrimaryAlt : deathMetal , neutralSecondary : heavyMetal , neutralTertiary : indieRock , neutralTertiaryAlt : punkRock , neutralLight : altRock , neutralLighter : popRock , redDark : dracula , red : lestat , white , } ; export const semanticColors < ISemanticColors > = { bodyText : palette . neutralPrimary , disabledBodyText : palette . neutralTertiaryAlt , bodySubtext : palette . neutralSecondary , buttonText : palette . neutralPrimaryAlt , errorText : palette . red , link : palette . themeDark , } ;	O O $any$ O $any$ O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O $Partial$ O $any$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O O O $Partial$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O
export enum TextColor { PRIMARY = 's' , SECONDARY = 's' , METADATA = 's' , WHITE = 's' , ERROR = 's' , DISABLED = 's' , } export enum TextSize { XXLARGE = 's' , XLARGE = 's' , LARGE = 's' , MEDIUM = 's' , MEDIUM_SUB = 's' , SMALL = 's' , XSMALL = 's' , }	O O $any$ O $TextColor.PRIMARY$ O O O $TextColor.SECONDARY$ O O O $TextColor.METADATA$ O O O $TextColor.WHITE$ O O O $TextColor.ERROR$ O O O $TextColor.DISABLED$ O O O O O O $any$ O $TextSize.XXLARGE$ O O O $TextSize.XLARGE$ O O O $TextSize.LARGE$ O O O $TextSize.MEDIUM$ O O O $TextSize.MEDIUM_SUB$ O O O $TextSize.SMALL$ O O O $TextSize.XSMALL$ O O O O
import { palette , semanticColors } from 's' ; import { TextSize , TextColor } from 's' ; import { IRawStyle , IFontWeight } from 's' ; export type Sizes = { [ size in TextSize ] : string } ; export const fontSizes = { xSmall : 's' , small : 's' , mediumSub : 's' , medium : 's' , large : 's' , xLarge : 's' , xxLarge : 's' , } ; export const lineHeights = { xSmall : 's' , small : 's' , mediumSub : 's' , medium : 's' , large : 's' , xLarge : 's' , xxLarge : 's' , } ; export const verticalAligns = { xSmall : 's' , small : 's' , mediumSub : 's' , medium : 's' , large : 's' , xLarge : 's' , xxLarge : 's' , } ; export const textColors : { [ color in TextColor ] : string | undefined } = { primary : semanticColors . bodyText , secondary : palette . neutralPrimaryAlt , metadata : palette . neutralSecondary , white : palette . white , error : palette . redDark , disabled : semanticColors . disabledBodyText , } ; export const fontWeights : { [ weight in 's' | 's' ] : IFontWeight } = { normal : 's' , bold : 's' , } ; export const ellipsisStyle = { overflow : 's' , textOverflow : 's' , whiteSpace : 's' , } ;	O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O O O $any$ O $any$ O O O O O O O $Sizes$ O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O O O O O $Sizes$ O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O O O O O $Sizes$ O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O O O O O $complex$ O O O $any$ O $any$ O O O O O O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O $complex$ O O O $any$ O O O O O O $any$ O O O $string$ O O O $string$ O O O O O O O $IRawStyle$ O O $string$ O O O $string$ O O O $string$ O O O O O
import { GutterSize } from 's' ; export const gutterSize : { [ size in GutterSize ] : string } = { none : 's' , xSmall : 's' , small : 's' , medium : 's' , large : 's' , xLarge : 's' , xxLarge : 's' , } ;	O O $any$ O O O O O O $complex$ O O O $any$ O $any$ O O O O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O O O
import { BlockProps } from 's' ; import { GutterSize } from 's' ; import { textColors , fontSizes , lineHeights , ellipsisStyle } from 's' ; import { gutterSize } from 's' ; import { IRawStyle } from 's' ; const getMarginTop = ( topSpacing ? , push ? ) => { if ( push ) { return push < 0 ? `template` : undefined ; } if ( topSpacing ) { return gutterSize [ topSpacing ] ; } } ; export const getStyles = ( props : BlockProps & { withinClickable : boolean } ) : IRawStyle => { const { push , textSize , topSpacing , bottomSpacing , textAlign , textColor , withinClickable } = props ; return { textAlign , width : withinClickable ? 's' : undefined , color : textColor ? textColors [ textColor ] : undefined , fontSize : textSize ? fontSizes [ textSize ] : undefined , lineHeight : textSize ? lineHeights [ textSize ] : undefined , marginTop : getMarginTop ( topSpacing , push ) , marginBottom : bottomSpacing ? gutterSize [ bottomSpacing ] : undefined , paddingTop : push && push > 0 ? `template` : undefined , } ; } ; export const getInnerStyles = ( props ) : IRawStyle => { const { padding , horizontalPadding , verticalPadding , ellipsis } = props ; return { ... ( ellipsis ? ellipsisStyle : { } ) , padding : padding ? gutterSize [ padding ] : undefined , ... ( horizontalPadding ? { paddingLeft : gutterSize [ horizontalPadding ] , paddingRight : gutterSize [ horizontalPadding ] , } : { } ) , ... ( verticalPadding ? { paddingTop : gutterSize [ verticalPadding ] , paddingBottom : gutterSize [ verticalPadding ] , } : { } ) , wordWrap : 's' as 's' , overflowWrap : 's' as 's' , } ; } ;	O O $any$ O O O O O O $any$ O O O O O O $complex$ O $Sizes$ O $Sizes$ O $any$ O O O O O O $complex$ O O O O O O $any$ O O O O O $string$ O O $GutterSize$ $GutterSize$ O $number$ $number$ O O O O O $number$ O O O $number$ O O O O O $undefined$ O O O O $GutterSize$ O O O $complex$ O $GutterSize$ O O O O O O O $any$ O O $any$ O $any$ O O $boolean$ O O O O O $any$ O O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O $any$ O O O $any$ O $string$ O $any$ O O O $undefined$ O $any$ O $any$ O $complex$ O $any$ O O $undefined$ O $any$ O $any$ O $Sizes$ O $any$ O O $undefined$ O $any$ O $any$ O $Sizes$ O $any$ O O $undefined$ O $string$ O $string$ O $any$ O $any$ O O $any$ O $any$ O $complex$ O $any$ O O $undefined$ O $string$ O $any$ O $any$ O O O O O $undefined$ O O O O O O O $any$ O O $BlockProps$ O O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O $complex$ O $any$ O O $undefined$ O O O $any$ O O $any$ O $complex$ O $any$ O O $any$ O $complex$ O $any$ O O O O O O O O O O $any$ O O $any$ O $complex$ O $any$ O O $any$ O $complex$ O $any$ O O O O O O O O O O O O O O O O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum ButtonColor { PRIMARY = 's' , SECONDARY = 's' , } export enum ButtonStatus { ENABLED = 's' , DISABLED = 's' , LOADING = 's' , } export enum ButtonIconPosition { LEFT = 's' , RIGHT = 's' , } export enum ButtonSize { REGULAR = 's' , SMALL = 's' , } export enum ButtonType { BUTTON = 's' , SUBMIT = 's' , RESET = 's' , }	O O $any$ O $ButtonColor.PRIMARY$ O O O $ButtonColor.SECONDARY$ O O O O O O $any$ O $ButtonStatus.ENABLED$ O O O $ButtonStatus.DISABLED$ O O O $ButtonStatus.LOADING$ O O O O O O $any$ O $ButtonIconPosition.LEFT$ O O O $ButtonIconPosition.RIGHT$ O O O O O O $any$ O $ButtonSize.REGULAR$ O O O $ButtonSize.SMALL$ O O O O O O $any$ O $ButtonType.BUTTON$ O O O $ButtonType.SUBMIT$ O O O $ButtonType.RESET$ O O O O
export const borderRadiusSoft = 's' ; export const borderRadiusRound = 's' ;	O O O O O O O O O O O O
import { palette } from 's' ; import { IButtonStyles } from 's' ; import { ButtonColor , ButtonSize , ButtonStatus } from 's' ; import { ButtonProps } from 's' ; import { fontWeights } from 's' ; import { borderRadiusSoft } from 's' ; const smallPadding = 's' ; const regularPadding = 's' ; const primaryColors = { root : { color : palette . white , backgroundColor : palette . themeDark , borderColor : palette . themeDark , } , rootHovered : { color : palette . white , backgroundColor : palette . themeDarkAlt , borderColor : palette . themeDarkAlt , textDecoration : 's' , } , rootFocused : { color : palette . white , backgroundColor : palette . themeDarkAlt , borderColor : palette . themeDarkAlt , } , rootPressed : { color : palette . white , backgroundColor : palette . themeDarker , borderColor : palette . themeDarker , } , } ; const secondaryColors = { root : { color : palette . themeDark , backgroundColor : palette . white , borderColor : palette . neutralDark , } , rootHovered : { color : palette . themeDark , backgroundColor : palette . white , borderColor : palette . themeLighter , textDecoration : 's' , } , rootFocused : { color : palette . themeDark , backgroundColor : palette . white , borderColor : palette . themeLighter , } , rootPressed : { color : palette . themeDark , backgroundColor : palette . white , borderColor : palette . themePrimary , } , } ; export const getStyles = ( props ) : IButtonStyles => { const { size , color , status , fullWidth } = props ; const colors = color === ButtonColor . PRIMARY ? primaryColors : secondaryColors ; const { root , rootHovered , rootFocused , rootPressed } = colors ; return { root : { ... ( root as { } ) , borderRadius : borderRadiusSoft , fontWeight : fontWeights . bold , textAlign : 's' , display : fullWidth ? 's' : 's' , position : 's' , whiteSpace : 's' , transition : 's' , borderWidth : 's' , borderStyle : 's' , boxShadow : 's' , padding : size === ButtonSize . REGULAR ? regularPadding : smallPadding , cursor : status === ButtonStatus . ENABLED ? 's' : 's' , opacity : status === ButtonStatus . DISABLED ? 's' : 's' , width : fullWidth ? 's' : undefined , } , rootHovered : { ... ( rootHovered as { } ) , } , rootFocused : { ... ( rootFocused as { } ) , } , rootPressed : { ... ( rootPressed as { } ) , } , } ; } ;	O O $any$ O O O O O O $any$ O O O O O O $any$ O $any$ O $any$ O O O O O O $any$ O O O O O O $complex$ O O O O O O O O O O O O O O O O O O O O O O $IButtonStyles$ O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $string$ O O O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O O $IButtonStyles$ O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $string$ O O O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O O O O O $any$ O O $ButtonProps$ O O $any$ O O O O $any$ O $any$ O $any$ O $any$ O O $any$ O O $any$ O $any$ O $any$ O $ButtonColor.PRIMARY$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O $any$ O O $any$ O O O $complex$ O O O O $any$ O O O O O $string$ O O O $any$ O $complex$ O $any$ O $string$ O O O $string$ O $any$ O O O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $string$ O $any$ O $any$ O $ButtonSize.REGULAR$ O O O O O $string$ O $any$ O $any$ O $ButtonStatus.ENABLED$ O O O O O $string$ O $any$ O $any$ O $ButtonStatus.DISABLED$ O O O O O $string$ O $any$ O O O $undefined$ O O O ${}$ O O O O $any$ O O O O O O O ${}$ O O O O $any$ O O O O O O O ${}$ O O O O $any$ O O O O O O O O O O O
export { default } from 's' ; export * from 's' ; export { default as BistateButton } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
import { loadTheme } from 's' ; import { palette , semanticColors } from 's' ; loadTheme ( { palette , semanticColors , fonts : { xSmall : { fontSize : 's' , lineHeight : 's' , } , small : { fontSize : 's' , lineHeight : 's' , } , smallPlus : { fontSize : 's' , lineHeight : 's' , } , medium : { fontSize : 's' , lineHeight : 's' , } , large : { fontSize : 's' , lineHeight : 's' , } , xLarge : { fontSize : 's' , lineHeight : 's' , } , xxLarge : { fontSize : 's' , lineHeight : 's' , } , } , } ) ;	O O $any$ O O O O O O $any$ O $any$ O O O O $any$ O O $any$ O $any$ O $complex$ O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O O O O O O O O
import { KeyCodes } from 's' ; export { KeyCodes } ; export const directionalKeyCodes = [ KeyCodes . up , KeyCodes . down , KeyCodes . left , KeyCodes . right , KeyCodes . home , KeyCodes . end , KeyCodes . tab , KeyCodes . pageUp , KeyCodes . pageDown , ] ;	O O $any$ O O O O O O $any$ O O O O $any[]$ O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O O
import { EventGroup } from 's' ; import { directionalKeyCodes } from 's' ; export type NavigationMode = 's' | 's' ; export type NavigationModeCallback = ( mode ) => void ; export default class KeyboardNavigationEventListener { private element : HTMLElement ; private eventGroup : EventGroup ; private mode : NavigationMode ; constructor ( element ) { this . element = element ; this . mode = 's' ; this . eventGroup = new EventGroup ( this ) ; this . eventGroup . on ( this . element , 's' , this . onMouseDown , true ) ; this . eventGroup . on ( this . element , 's' , this . onKeyDown , true ) ; } public getMode ( ) { return this . mode ; } public destroy ( ) { this . eventGroup . off ( ) ; } public subscribe ( callback ) { this . eventGroup . on ( this , 's' , callback ) ; } public unsubscribe ( callback ) { this . eventGroup . off ( this , 's' , callback ) ; } private onMouseDown = ( ) => { if ( this . mode !== 's' ) { this . mode = 's' ; this . eventGroup . raise ( 's' , this . mode ) ; } } ; private onKeyDown = ( event ) => { if ( directionalKeyCodes . indexOf ( event . which ) > - 0 ) { this . mode = 's' ; this . eventGroup . raise ( 's' , this . mode ) ; } } ; }	O O $any$ O O O O O O $any[]$ O O O O O O $any$ O O O O O O O $any$ O O $NavigationMode$ O O O O O O O $any$ O O $HTMLElement$ O $complex$ O O $any$ O $any$ O O $NavigationMode$ O $any$ O O O $HTMLElement$ O O O O $HTMLElement$ O $HTMLElement$ O O O $NavigationMode$ O O O O O $any$ O O $any$ O O O O O O $any$ O $any$ O O O $HTMLElement$ O O O O O $void$ O O O O O O $any$ O $any$ O O O $HTMLElement$ O O O O O $void$ O O O O O O $NavigationMode$ O O O O O O $NavigationMode$ O O O $void$ O O O O O $any$ O $any$ O O O O O $void$ O $NavigationModeCallback$ O O O O $any$ O $any$ O O O O O $NavigationModeCallback$ O O O O $void$ O $NavigationModeCallback$ O O O O $any$ O $any$ O O O O O $NavigationModeCallback$ O O O O $void$ O O O O O O O O O $NavigationMode$ O O O O O O $NavigationMode$ O O O O O $any$ O $any$ O O O O O O O O O O O O $void$ O O $KeyboardEvent$ O O O O O $any[]$ O $number$ O $KeyboardEvent$ O $number$ O O O O O O O O $NavigationMode$ O O O O O $any$ O $any$ O O O O O O O O O O O O
import KeyboardNavigationEventListener , { NavigationMode , NavigationModeCallback , } from 's' ; export { NavigationMode , NavigationModeCallback } ; let navListener ; const ensureInit = ( ) => { if ( ! navListener ) { navListener = new KeyboardNavigationEventListener ( document . body ) ; } } ; export const getMode = ( ) : NavigationMode => { ensureInit ( ) ; return navListener . getMode ( ) ; } ; export const subscribe = ( callback ) : void => { ensureInit ( ) ; navListener . subscribe ( callback ) ; } ; export const unsubscribe = ( callback ) : void => { ensureInit ( ) ; navListener . unsubscribe ( callback ) ; } ;	O $any$ O O $any$ O $any$ O O O O O O O $any$ O $any$ O O O $KeyboardNavigationEventListener$ O O $void$ O O O O O O O O $KeyboardNavigationEventListener$ O O $KeyboardNavigationEventListener$ O O $any$ O $Document$ O $HTMLElement$ O O O O O O O $NavigationMode$ O O O O $any$ O O $void$ O O O O $KeyboardNavigationEventListener$ O $NavigationMode$ O O O O O O O $void$ O O $NavigationModeCallback$ O O O O O $void$ O O O $KeyboardNavigationEventListener$ O $void$ O $NavigationModeCallback$ O O O O O O $void$ O O $NavigationModeCallback$ O O O O O $void$ O O O $KeyboardNavigationEventListener$ O $void$ O $NavigationModeCallback$ O O O O
import 's' ; import 's' ;	O O O O O O
import 's' ; import { Fabric } from 's' ; export default Fabric ;	O O O O O $any$ O O O O O O $any$ O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default as FixedGridRow } from 's' ; export * from 's' ; export { default as FixedGridColumn } from 's' ; export * from 's' ;	O O $any$ O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export type HeadingLevel = 's' | 's' | 's' | 's' | 's' | 's' ; export type HeadingSize = HeadingLevel | 's' ;	O O $any$ O O O O O O O O O O O O O O O $any$ O $any$ O O O
export { default as Hovercard } from 's' ; export * from 's' ; export { default as HovercardHeader } from 's' ; export * from 's' ; export { default as HovercardBody } from 's' ; export * from 's' ;	O O $any$ O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O
export { default as Accounts } from 's' ; export { default as ActivityFeed } from 's' ; export { default as Add } from 's' ; export { default as AddFavorite } from 's' ; export { default as AddFriend } from 's' ; export { default as Asterisk } from 's' ; export { default as Attach } from 's' ; export { default as Back } from 's' ; export { default as BadgeAdmin } from 's' ; export { default as BadgeFlexternal } from 's' ; export { default as BadgeNew } from 's' ; export { default as 0 } from 's' ; export { default as BarChartHorizontal } from 's' ; export { default as Blocked } from 's' ; export { default as BulletedList } from 's' ; export { default as Calendar } from 's' ; export { default as Camera } from 's' ; export { default as Cancel } from 's' ; export { default as CannedChat } from 's' ; export { default as CellPhone } from 's' ; export { default as Checkbox } from 's' ; export { default as CheckboxComposite } from 's' ; export { default as CheckMark } from 's' ; export { default as CheckMarkEmpty } from 's' ; export { default as ChevronDownMed } from 's' ; export { default as ChevronLeftMed } from 's' ; export { default as ChevronRightMed } from 's' ; export { default as ChevronUpMed } from 's' ; export { default as CircleCheck } from 's' ; export { default as CircleFill } from 's' ; export { default as CircleRing } from 's' ; export { default as Clock } from 's' ; export { default as Code } from 's' ; export { default as Connectors } from 's' ; export { default as Contact } from 's' ; export { default as Delete } from 's' ; export { default as DietPlanNotebook } from 's' ; export { default as Down } from 's' ; export { default as Download } from 's' ; export { default as DynamicGroup } from 's' ; export { default as Edit } from 's' ; export { default as EditEvent } from 's' ; export { default as Embed } from 's' ; export { default as EMI } from 's' ; export { default as EngineeringGroup } from 's' ; export { default as FabricFolder } from 's' ; export { default as FavoriteStar } from 's' ; export { default as FavoriteStarFill } from 's' ; export { default as FavoriteStartRemove } from 's' ; export { default as FileBug } from 's' ; export { default as Filter } from 's' ; export { default as Flag } from 's' ; export { default as Forward } from 's' ; export { default as ForwardAlt } from 's' ; export { default as GIF } from 's' ; export { default as Globe } from 's' ; export { default as Group } from 's' ; export { default as Hashtag } from 's' ; export { default as Home } from 's' ; export { default as Info } from 's' ; export { default as Like } from 's' ; export { default as LikeSolid } from 's' ; export { default as Link } from 's' ; export { default as LiveEvent } from 's' ; export { default as LocationDot } from 's' ; export { default as Lock } from 's' ; export { default as Mail } from 's' ; export { default as Megaphone } from 's' ; export { default as More } from 's' ; export { default as MoreVertical } from 's' ; export { default as NavigateExternalInline } from 's' ; export { default as NumberedList } from 's' ; export { default as OneDrive } from 's' ; export { default as OneNoteLogoInverse } from 's' ; export { default as People } from 's' ; export { default as Phone } from 's' ; export { default as Pinned } from 's' ; export { default as PlannerLogo } from 's' ; export { default as Play } from 's' ; export { default as Puzzle } from 's' ; export { default as QuestionMessage } from 's' ; export { default as Refresh } from 's' ; export { default as Remove } from 's' ; export { default as RemoveEvent } from 's' ; export { default as RemoveLink } from 's' ; export { default as Reply } from 's' ; export { default as Ringer } from 's' ; export { default as ScaleUp } from 's' ; export { default as ScrollUpDown } from 's' ; export { default as Search } from 's' ; export { default as SemiBoldWeight } from 's' ; export { default as Send } from 's' ; export { default as Settings } from 's' ; export { default as Share } from 's' ; export { default as SharePointLogoInverse } from 's' ; export { default as Shield } from 's' ; export { default as SignOut } from 's' ; export { default as SingleBookmark } from 's' ; export { default as SkypeCircleCheck } from 's' ; export { default as SkypeCircleClock } from 's' ; export { default as SkypeCircleMinus } from 's' ; export { default as SkypeForBusinessLogo } from 's' ; export { default as SortDown } from 's' ; export { default as SortUp } from 's' ; export { default as Spam } from 's' ; export { default as Strikethrough } from 's' ; export { default as Tag } from 's' ; export { default as 0 } from 's' ; export { default as Underline } from 's' ; export { default as Undo } from 's' ; export { default as Unknown } from 's' ; export { default as Up } from 's' ; export { default as Upload } from 's' ; export { default as UserFollowed } from 's' ; export { default as UserRemove } from 's' ; export { default as Video } from 's' ; export { default as View } from 's' ; export { default as Warning } from 's' ; export { default as Work } from 's' ; export { default as World } from 's' ; export { default as Yammer } from 's' ; export { default as ZoomIn } from 's' ;	O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O O O $any$ O $any$ O O O O
export enum IconSize { XXLARGE = 's' , XLARGE = 's' , LARGE = 's' , MEDIUM = 's' , SMALL = 's' , XSMALL = 's' , }	O O $any$ O $IconSize.XXLARGE$ O O O $IconSize.XLARGE$ O O O $IconSize.LARGE$ O O O $IconSize.MEDIUM$ O O O $IconSize.SMALL$ O O O $IconSize.XSMALL$ O O O O
export * from 's' ; export { default as BaseIcon , IconProps } from 's' ;	O O O O O O O $any$ O $any$ O $any$ O O O O"""
train_data_examples = train_data.splitlines()
token_vocab = set()
tag_vocab = set()

def preprocess_line(line):
    line = line.split('\t')
    tokens = line[0].split(' ')
    tags = line[1].split(' ')
    return (tokens, tags)

# start_tags[i] = prob(sentence starts with tag i)
# transitions[j][i] = prob(tag i given tag j was previous)
# emissions[j][i] = prob(token i given tag j)
start_tags = defaultdict(float)
transitions = defaultdict(lambda: defaultdict(float))
emissions = defaultdict(lambda: defaultdict(float))
for example in train_data_examples:
    tokens, tags = preprocess_line(example)
    assert(len(tokens) == len(tags))

    # update counters
    token_vocab = token_vocab | set(tokens)
    tag_vocab = tag_vocab | set(tags)
    start_tags[tags[0]] += 1
    for i in range(len(tags) - 1):
        transitions[tags[i]][tags[i + 1]] += 1
        emissions[tags[i]][tokens[i]] += 1
    emissions[tags[len(tags) - 1]][tokens[len(tags) - 1]] += 1

token_vocab.add('<unk>')

token_vocab = sorted(token_vocab)
tag_vocab = sorted(tag_vocab)

print('token vocab length:', len(token_vocab))
print('tag vocab length:', len(tag_vocab))

token vocab length: 552
tag vocab length: 75


In [116]:
import numpy as np

# add-0.01 smoothing
for tag in tag_vocab:
    start_tags[tag] += 0.01
    for prev_tag in tag_vocab:
        transitions[prev_tag][tag] += 0.01
    for token in token_vocab:
        emissions[tag][token] += 0.01

SCALE_HYPERPARAM = 0.1

# reduce probabilities of transitioning to 'O'
start_tags['O'] *= SCALE_HYPERPARAM
for prev_tag in tag_vocab:
    transitions[prev_tag]['O'] *= SCALE_HYPERPARAM

# normalize HMM probability tables
start_tags_total = sum([start_tags[tag] for tag in tag_vocab])
for tag, count in start_tags.items():
    start_tags[tag] /= start_tags_total

for prev_tag in tag_vocab:
    total = sum([transitions[prev_tag][tag] for tag in tag_vocab])
    for tag in tag_vocab:
        transitions[prev_tag][tag] /= total

for tag in tag_vocab:
    total = sum([emissions[tag][token] for token in token_vocab])
    for token in token_vocab:
        emissions[tag][token] /= total

# convert sparse dictionaries into vectors
start_probs = np.array([start_tags[tag] for tag in tag_vocab])
transition_probs = np.array([[transitions[prev_tag][tag] for tag in tag_vocab] for prev_tag in tag_vocab])
emission_probs = np.array([[emissions[tag][token] for token in token_vocab] for tag in tag_vocab])

print(token_vocab)
print(tag_vocab)
# print(start_probs)
# print(transition_probs)
# print(emission_probs)

assert(abs(sum(start_probs) - 1) < 0.01)
assert(all([abs(sum(row) - 1) < 0.01 for row in transition_probs]))
assert(all([abs(sum(row) - 1) < 0.01 for row in emission_probs]))

['!', '!==', '"s"', '%', '%=', '&', '&&', "'js'", "'s'", '(', ')', '*', ',', '-', '.', '...', '0', ':', ';', '<', '<unk>', '=', '===', '=>', '>', '>=', '?', 'Accounts', 'ActivityFeed', 'Adapter', 'Add', 'AddFavorite', 'AddFriend', 'Asterisk', 'Attach', 'AvatarSize', 'BLACK', 'BUTTON', 'Back', 'BadgeAdmin', 'BadgeFlexternal', 'BadgeNew', 'BarChartHorizontal', 'BaseIcon', 'BistateButton', 'BlockProps', 'Blocked', 'BulletedList', 'ButtonColor', 'ButtonIconPosition', 'ButtonProps', 'ButtonSize', 'ButtonStatus', 'ButtonType', 'Calendar', 'Camera', 'Cancel', 'CannedChat', 'CellPhone', 'CheckMark', 'CheckMarkEmpty', 'Checkbox', 'CheckboxComposite', 'ChevronDownMed', 'ChevronLeftMed', 'ChevronRightMed', 'ChevronUpMed', 'CircleCheck', 'CircleFill', 'CircleRing', 'Clock', 'Code', 'Connectors', 'Contact', 'DISABLED', 'Delete', 'DietPlanNotebook', 'Down', 'Download', 'DynamicGroup', 'EMI', 'ENABLED', 'ERROR', 'Edit', 'EditEvent', 'Embed', 'EngineeringGroup', 'Error', 'EventGroup', 'Fabric', 'Fabri

In [117]:
import numpy as np
from hmmlearn import hmm

test_data = """export { default as BaseIllustration , IllustrationProps } from 's' ;	O O $any$ O $any$ O $any$ O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum BorderType { SOFT = 's' , ROUND = 's' , }	O O $any$ O $BorderType.SOFT$ O O O $BorderType.ROUND$ O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default as LayoutList } from 's' ; export * from 's' ; export { default as LayoutListItem } from 's' ; export * from 's' ;	O O $any$ O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O
export { default as List } from 's' ; export * from 's' ; export { default as ListItem } from 's' ; export * from 's' ;	O O $any$ O $any$ O O O O O O O O O O O $any$ O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum MediaObjectSize { XLARGE = 's' , LARGE = 's' , MEDIUM = 's' , SMALL = 's' , XSMALL = 's' , }	O O $any$ O $MediaObjectSize.XLARGE$ O O O $MediaObjectSize.LARGE$ O O O $MediaObjectSize.MEDIUM$ O O O $MediaObjectSize.SMALL$ O O O $MediaObjectSize.XSMALL$ O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
import { ContextualMenuItemType } from 's' ; export enum MenuItemType { Normal = ContextualMenuItemType . Normal , Header = ContextualMenuItemType . Header , Divider = ContextualMenuItemType . Divider , }	O O $any$ O O O O O O $any$ O $MenuItemType$ O $any$ O $any$ O $MenuItemType$ O $any$ O $any$ O $MenuItemType$ O $any$ O $any$ O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum MessageBarType { INFO = 's' , WARNING = 's' , ERROR = 's' , }	O O $any$ O $MessageBarType.INFO$ O O O $MessageBarType.WARNING$ O O O $MessageBarType.ERROR$ O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export enum SpinnerSize { LARGE = 's' , MEDIUM = 's' , SMALL = 's' , XSMALL = 's' , }	O O $any$ O $SpinnerSize.LARGE$ O O O $SpinnerSize.MEDIUM$ O O O $SpinnerSize.SMALL$ O O O $SpinnerSize.XSMALL$ O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
import { TextProps , TextSize } from 's' ; import { ellipsisStyle , fontWeights , fontSizes , lineHeights , textColors , verticalAligns , } from 's' ; import { IRawStyle } from 's' ; const getHeight = ( size ? ) => { return size ? lineHeights [ size ] : undefined ; } ; const getVerticalAlignForMaxWidth = ( size ? ) => { return size ? verticalAligns [ size ] : 's' ; } ; export const getStyles = ( props : TextProps & { blockTextSize ? : TextSize } ) : IRawStyle => { const { size , maxWidth , bold , uppercase , color , blockTextSize } = props ; return { ... ( maxWidth ? ellipsisStyle : { } ) , fontWeight : bold ? fontWeights . bold : undefined , display : 's' , textTransform : uppercase ? 's' : undefined , fontSize : size ? fontSizes [ size ] : undefined , lineHeight : size ? lineHeights [ size ] : undefined , color : color ? textColors [ color ] : undefined , maxWidth : maxWidth || undefined , height : maxWidth ? getHeight ( size ) : undefined , verticalAlign : maxWidth ? getVerticalAlignForMaxWidth ( size || blockTextSize ) : undefined , } ; } ;	O O $any$ O $any$ O O O O O O $any$ O $complex$ O $Sizes$ O $Sizes$ O $complex$ O $Sizes$ O O O O O O O $any$ O O O O O $any$ O O $any$ $TextSize$ O O O O $any$ O $Sizes$ O $any$ O O $undefined$ O O O O $any$ O O $any$ $TextSize$ O O O O $any$ O $Sizes$ O $any$ O O O O O O O O $any$ O O $any$ O $any$ O O $any$ O O $any$ O O O $any$ O O O O $any$ O $any$ O $any$ O $any$ O $any$ O $any$ O O $any$ O O O O O $any$ O $any$ O O O O O $any$ O $any$ O $complex$ O $any$ O $undefined$ O $string$ O O O $string$ O $any$ O O O $undefined$ O $any$ O $any$ O $Sizes$ O $any$ O O $undefined$ O $any$ O $any$ O $Sizes$ O $any$ O O $undefined$ O $any$ O $any$ O $complex$ O $any$ O O $undefined$ O $any$ O $any$ O $undefined$ O $any$ O $any$ O $any$ O $any$ O O $undefined$ O $any$ O $any$ O $any$ O $any$ O $any$ O O $undefined$ O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
import { palette } from 's' ; import { TooltipProps } from 's' ; import { lineHeights , fontSizes } from 's' ; import { borderRadiusSoft } from 's' ; import { ITooltipStyles } from 's' ; import { ICalloutContentStyles } from 's' ; export const getTooltipStyles = ( _ ) : ITooltipStyles => { return { root : { lineHeight : 's' , } , content : { display : 's' , } , subText : { fontSize : fontSizes . small , lineHeight : lineHeights . small , color : palette . white , } , } ; } ; export const getCalloutStyles = ( _ ) : ICalloutContentStyles => { return { container : { backgroundColor : 's' , } , root : { backgroundColor : 's' , boxShadow : 's' , padding : 's' , minHeight : 's' , zIndex : 0 , border : 's' , } , beak : { backgroundColor : palette . neutralPrimary , } , beakCurtain : { backgroundColor : palette . neutralPrimary , borderRadius : borderRadiusSoft , } , calloutMain : { backgroundColor : palette . neutralPrimary , } , } ; } ;	O O $any$ O O O O O O $any$ O O O O O O $Sizes$ O $Sizes$ O O O O O O O O O O O O O $any$ O O O O O O $any$ O O O O O O $any$ O O $TooltipProps$ O O $any$ O O O O $complex$ O O $string$ O O O O O $complex$ O O $string$ O O O O O $complex$ O O $string$ O $Sizes$ O $string$ O $string$ O $Sizes$ O $string$ O $any$ O $any$ O $any$ O O O O O O O O O $any$ O O $TooltipProps$ O O $any$ O O O O $complex$ O O $string$ O O O O O $complex$ O O $string$ O O O $string$ O O O $string$ O O O $string$ O O O $number$ O O O $string$ O O O O O $complex$ O O $any$ O $any$ O $any$ O O O $complex$ O O $any$ O $any$ O $any$ O $string$ O O O O O $complex$ O O $any$ O $any$ O $any$ O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export { default } from 's' ; export * from 's' ;	O O $any$ O O O O O O O O O
export interface Focusable { focus ( ) : void ; }	O O $any$ O $void$ O O O O O O
import { Focusable } from 's' ; export interface BaseComponentProps { className ? : string ; } export interface NestableBaseComponentProps extends BaseComponentProps { children ? : React . ReactNode ; } export interface FocusableComponentProps { focusableRef ? ( ref : Focusable | null ) : void ; }	O O $any$ O O O O O O $any$ O $string$ O O O O O O O $any$ O $any$ O $any$ O O $any$ O $any$ O O O O $any$ O $void$ O O $Focusable$ O $any$ O O O O O O O
import KeyboardNavigationEventListener from 's' ; describe ( 's' , ( ) => { let listener ; let 0 : jest . Mock ; let 0 : jest . Mock ; beforeEach ( ( ) => { listener = new KeyboardNavigationEventListener ( document . body ) ; 0 = jest . fn ( ) ; 0 = jest . fn ( ) ; listener . subscribe ( 0 ) ; listener . subscribe ( 0 ) ; } ) ; afterEach ( ( ) => { listener . destroy ( ) ; jest . resetAllMocks ( ) ; } ) ; it ( 's' , ( ) => { expect ( listener . getMode ( ) ) . toBe ( 's' ) ; } ) ; } ) ;	O $any$ O O O $any$ O O O O O O O O $KeyboardNavigationEventListener$ O O $any$ O $any$ O $any$ O O $any$ O $any$ O $any$ O $any$ O O O O O $KeyboardNavigationEventListener$ O O $any$ O $Document$ O $HTMLElement$ O O $any$ O $any$ O $any$ O O O $any$ O $any$ O $any$ O O O $KeyboardNavigationEventListener$ O $void$ O $any$ O O $KeyboardNavigationEventListener$ O $void$ O $any$ O O O O O $any$ O O O O O $KeyboardNavigationEventListener$ O $void$ O O O $any$ O $any$ O O O O O O $any$ O O O O O O O $any$ O $KeyboardNavigationEventListener$ O $NavigationMode$ O O O O $any$ O O O O O O O O O O
jest . mock ( 's' ) ; import KeyboardNavigationEventListener from 's' ; import { getMode , subscribe , unsubscribe } from 's' ; describe ( 's' , ( ) => { let callback : jest . Mock ; beforeEach ( ( ) => { callback = jest . fn ( ) ; } ) ; afterEach ( ( ) => { jest . resetAllMocks ( ) ; } ) ; describe ( 's' , ( ) => { beforeEach ( ( ) => { jest . spyOn ( KeyboardNavigationEventListener . prototype , 's' ) . mockReturnValue ( 's' ) ; } ) ; it ( 's' , ( ) => { expect ( getMode ( ) ) . toBe ( 's' ) ; } ) ; } ) ; describe ( 's' , ( ) => { beforeEach ( ( ) => { jest . spyOn ( KeyboardNavigationEventListener . prototype , 's' ) . mockImplementation ( cb => cb ( 's' ) ) ; subscribe ( callback ) ; } ) ; it ( 's' , ( ) => { expect ( callback ) . toHaveBeenCalledWith ( 's' ) ; } ) ; } ) ; describe ( 's' , ( ) => { beforeEach ( ( ) => { jest . spyOn ( KeyboardNavigationEventListener . prototype , 's' ) . mockImplementation ( cb => cb ( 's' ) ) ; unsubscribe ( callback ) ; } ) ; it ( 's' , ( ) => { expect ( callback ) . toHaveBeenCalledWith ( 's' ) ; } ) ; } ) ; } ) ;	$any$ O $any$ O O O O O $any$ O O O O O $NavigationMode$ O $void$ O $void$ O O O O $any$ O O O O O O O O $any$ O $any$ O $any$ O $any$ O O O O O $any$ O $any$ O $any$ O O O O O O $any$ O O O O O $any$ O $any$ O O O O O O $any$ O O O O O O O $any$ O O O O O $any$ O $any$ O $any$ O $KeyboardNavigationEventListener$ O O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $NavigationMode$ O O O O $any$ O O O O O O O O O O $any$ O O O O O O O $any$ O O O O O $any$ O $any$ O $any$ O $KeyboardNavigationEventListener$ O O O O $any$ O $any$ O $any$ O O O O O $void$ O $any$ O O O O O $any$ O O O O O O O $any$ O $any$ O O $any$ O O O O O O O O O O $any$ O O O O O O O $any$ O O O O O $any$ O $any$ O $any$ O $KeyboardNavigationEventListener$ O O O O $any$ O $any$ O $any$ O O O O O $void$ O $any$ O O O O O $any$ O O O O O O O $any$ O $any$ O O $any$ O O O O O O O O O O O O O
export type ClassName = string | undefined ; export const join = ( classNames : ClassName [ ] ) : string => { if ( classNames . length === 0 ) { return 's' ; } return classNames . filter ( className => ! ! className ) . join ( 's' ) ; } ;	O O $any$ O O O O O O O $string$ O O $string[]$ O $any$ O O O O O O O O O $string[]$ O $number$ O O O O O O O O O $string[]$ O $complex$ O $string$ O O O $string$ O O $string$ O O O O O O
import { join } from 's' ; describe ( 's' , ( ) => { const emptyArray : string [ ] = [ ] ; const oneStringItem = [ 's' ] ; const twoStringItems = [ 's' , 's' ] ; const threeStringItems = [ 's' , 's' , 's' ] ; const arrayWithEmptyItems = [ 's' , undefined , 's' ] ; describe ( 's' , ( ) => { it ( 's' , ( ) => { expect ( join ( emptyArray ) ) . toBe ( 's' ) ; } ) ; it ( 's' , ( ) => { expect ( join ( oneStringItem ) ) . toBe ( 's' ) ; } ) ; it ( 's' , ( ) => { expect ( join ( twoStringItems ) ) . toBe ( 's' ) ; } ) ; it ( 's' , ( ) => { expect ( join ( threeStringItems ) ) . toBe ( 's' ) ; } ) ; it ( 's' , ( ) => { expect ( join ( arrayWithEmptyItems ) ) . toBe ( 's' ) ; } ) ; } ) ; } ) ;	O O $string$ O O O O $any$ O O O O O O O O $string[]$ O O O O O O O O O $string[]$ O O O O O O $string[]$ O O O O O O O O $string[]$ O O O O O O O O O O $string[]$ O O O O $undefined$ O O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $string$ O $string[]$ O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $string$ O $string[]$ O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $string$ O $string[]$ O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $string$ O $string[]$ O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $string$ O $string[]$ O O O $any$ O O O O O O O O O O O O O
jest . mock ( 's' , ( ) => ( { loadTheme : jest . fn ( ) , } ) ) ; import * as Styling from 's' ; import 's' ; describe ( 's' , ( ) => { it ( 's' , ( ) => { expect ( Styling . loadTheme ) . toHaveBeenCalledTimes ( 0 ) ; } ) ; } ) ;	$any$ O $any$ O O O O O O O O $any$ O $any$ O $any$ O O O O O O O O O O $any$ O O O O O O $any$ O O O O O O O $any$ O O O O O O O $any$ O $any$ O $any$ O O $any$ O O O O O O O O O O
export type Diff < T extends string , U extends string > = ( { [ P in T ] : P } & { [ P in U ] : never } & { [ x ] : never } ) [ T ] ; export type Omit < T , K extends keyof T > = Pick < T , Diff < keyof T , K > > ;	O O $any$ O $any$ O O O $any$ O O O O O O O $any$ O $any$ O O $any$ O O O O $any$ O $any$ O O O O O O O $string$ O O O O O O $any$ O O O O $any$ O $any$ O $any$ O O $any$ O O $any$ O $any$ O $any$ O O $any$ O $any$ O O O"""
test_data_examples = test_data.splitlines()

model = hmm.MultinomialHMM(n_components=len(tag_vocab))
model.startprob_ = start_probs
model.transmat_ = transition_probs
model.emissionprob_ = emission_probs

unk_index = token_vocab.index('<unk>')
total_items_to_predict = 0.01  # prevent div by 0 if there is no test data
correctly_predicted = 0
for example in test_data_examples:
    tokens, tags = preprocess_line(example)
    token_indices = [int(token_vocab.index(token)) if token in token_vocab else unk_index for token in tokens]
    model_input = np.array([token_indices]).T
    pred_indices = model.predict(model_input)
    # print('real tags', tags)
    pred_tags = [tag_vocab[pred_indices[i]] for i in range(len(pred_indices))]
    # print('predicted tags', pred_tags)
    total_items_to_predict += sum([tag != 'O' for tag in tags])
    correctly_predicted += sum([real_tag == pred_tag for real_tag, pred_tag in zip(tags, pred_tags) if real_tag != 'O'])
    # print('accuracy', correctly_predicted / total_items_to_predict)

print('total accuracy', correctly_predicted / total_items_to_predict)

total accuracy 0.6821730911481877
